- We are given train.csv which has labels anf images folder which has images of pokemons

In [2]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from pathlib import Path
from keras.preprocessing import image

- Ignore the error. Or you can download 7gb CUDA toolkit to speed up if you have a dedicated GPU and ignore error if you have integrated graphics. 

### Load images which is our X_train

In [6]:
p = Path("./Train/Images")  ## create instance od Path to iterate in this directory
files = p.glob("*.jpg")  ## glob() method  returns a generator object to iterate over all files 
# in path passsed while creating object 'p'. We can use dirs generator in a for loop

image_data = []  ## create empty list to store all images in it
img_ids = []  ## id sfor each image is their name 
for file in files:  ## to iterate over all files matching with " *jpeg " regex in passed Path
    img = image.load_img(file, target_size=(32,32)) ## to load ith image
    img_array = image.img_to_array(img)  ## convert image to array
    image_data.append(img_array)
    
    img_id = str(file).split("/")[-1]
    img_ids.append(img_id)

In [8]:
image_data

[array([[[177., 159., 155.],
         [177., 162., 157.],
         [175., 158., 151.],
         ...,
         [197., 176., 173.],
         [197., 173., 173.],
         [195., 171., 169.]],
 
        [[174., 157., 150.],
         [179., 160., 156.],
         [177., 158., 152.],
         ...,
         [197., 173., 171.],
         [199., 175., 175.],
         [193., 172., 167.]],
 
        [[179., 162., 155.],
         [176., 159., 151.],
         [177., 159., 155.],
         ...,
         [194., 167., 172.],
         [189., 168., 163.],
         [193., 169., 159.]],
 
        ...,
 
        [[213., 191., 193.],
         [218., 196., 198.],
         [215., 194., 193.],
         ...,
         [213., 191., 193.],
         [209., 187., 189.],
         [212., 190., 192.]],
 
        [[212., 192., 193.],
         [212., 192., 193.],
         [216., 196., 197.],
         ...,
         [222., 195., 200.],
         [217., 195., 198.],
         [216., 196., 197.]],
 
        [[211., 191., 192.],
 

- These are all the images. Let us convert it's range in 0-1 for better results and use these pixel intensity as floating points which ranges from 0-1 and also convert this list to numpy array.

In [11]:
img_data = np.array(image_data, dtype="float32")/255.0
img_ids = np.array(img_ids)
print(img_data)

[[[[0.69411767 0.62352943 0.60784316]
   [0.69411767 0.63529414 0.6156863 ]
   [0.6862745  0.61960787 0.5921569 ]
   ...
   [0.77254903 0.6901961  0.6784314 ]
   [0.77254903 0.6784314  0.6784314 ]
   [0.7647059  0.67058825 0.6627451 ]]

  [[0.68235296 0.6156863  0.5882353 ]
   [0.7019608  0.627451   0.6117647 ]
   [0.69411767 0.61960787 0.59607846]
   ...
   [0.77254903 0.6784314  0.67058825]
   [0.78039217 0.6862745  0.6862745 ]
   [0.75686276 0.6745098  0.654902  ]]

  [[0.7019608  0.63529414 0.60784316]
   [0.6901961  0.62352943 0.5921569 ]
   [0.69411767 0.62352943 0.60784316]
   ...
   [0.7607843  0.654902   0.6745098 ]
   [0.7411765  0.65882355 0.6392157 ]
   [0.75686276 0.6627451  0.62352943]]

  ...

  [[0.8352941  0.7490196  0.75686276]
   [0.85490197 0.76862746 0.7764706 ]
   [0.84313726 0.7607843  0.75686276]
   ...
   [0.8352941  0.7490196  0.75686276]
   [0.81960785 0.73333335 0.7411765 ]
   [0.83137256 0.74509805 0.7529412 ]]

  [[0.83137256 0.7529412  0.75686276]
   [0.8

In [13]:
print(np.max(img_data), np.min(img_data))

1.0 0.0


- Now our intensity range 0.1-1.1 instead of 0-255
- And we will now flatten our image:

In [16]:
## see what img ids are:
print(img_ids[:5])  ## print first 5 ids
print(img_ids.shape)

['478.jpg' '463.jpg' '405.jpg' '458.jpg' '426.jpg']
(304,)


In [19]:
print("Image Size initially: ", np.array(image_data).shape) #image_data is list obtained initially
img_data = img_data.reshape((img_data.shape[0],-1))
print("Image Size After flattening image: ", img_data.shape)

Image Size initially:  (304, 32, 32, 3)
Image Size After flattening image:  (304, 3072)


- We can add ids as another feature to our column: making a total of 3073 columns where 1st feature = "id of image" and rest 3072 features are pixel intensities in float (0-1) range.

In [21]:
new_img_data = np.hstack((img_ids.reshape((-1,1)), img_data))
new_img_data.shape

(304, 3073)

In [23]:
new_img_data[0,:5]  ## print first 5 features for 1st image

array(['478.jpg', '0.69411767', '0.62352943', '0.60784316', '0.69411767'],
      dtype='<U32')

- for oth image: 1st feature= image_id and next features are pixel intensities

### Load y_labels which are given in trian.csv

In [25]:
labels = pd.read_csv("./Train/train.csv")
labels.head()

,ImageId,NameOfPokemon
0,296.jpg,Pikachu
1,297.jpg,Pikachu
2,298.jpg,Pikachu
3,299.jpg,Pikachu
4,300.jpg,Pikachu


In [27]:
print(labels.shape)
print(new_img_data.shape)

(304, 2)
(304, 3073)


- Labels has img_ids as their 1st column. So we can convert new_img_data array to a dataframe and merge them both **with respect to id column which is common in both arrarys**. Then we can split columns as X_train and y_train where name of pokemon is y_train and from x_train, we can remove img_id as img_id was only useful to match images with their labels.

In [37]:
data = pd.merge(pd.DataFrame(new_img_data), labels, left_on=0, right_on="ImageId")
data.head()

,0,1,2,3,4,5,6,7,8,9,...,3065,3066,3067,3068,3069,3070,3071,3072,ImageId,NameOfPokemon
0,478.jpg,0.69411767,0.62352943,0.60784316,0.69411767,0.63529414,0.6156863,0.6862745,0.61960787,0.5921569,...,0.7647059,0.7764706,0.85882354,0.7607843,0.7764706,0.85490197,0.75686276,0.76862746,478.jpg,Bulbasaur
1,463.jpg,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,463.jpg,Bulbasaur
2,405.jpg,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,405.jpg,Bulbasaur
3,458.jpg,0.5058824,0.5058824,0.5058824,0.5058824,0.5058824,0.5058824,0.5058824,0.5058824,0.5058824,...,0.5058824,0.5058824,0.5058824,0.5058824,0.5058824,0.5058824,0.5058824,0.5058824,458.jpg,Bulbasaur
4,426.jpg,0.24313726,0.38039216,0.21568628,0.23921569,0.3764706,0.21176471,0.23921569,0.3647059,0.20392157,...,0.62352943,0.3137255,0.4509804,0.7019608,0.33333334,0.41960785,0.654902,0.30980393,426.jpg,Bulbasaur


- Now we can split our data into X_train and y_train and we can drop column 0 and 2nd last: both are Image_ids bcoz we do not need these anymore.

In [39]:
X_train = data.values[:, 1:-2] ## except 1st column and last 2 column
y_train = data.values[: , -1]
X_train.shape, y_train.shape

((304, 3072), (304,))

- Now we have got X_trian and y_train 

### Training:

- If we are using our own svm from scratch, then we need to convert labels into integers and create a mapping for them with a dictionary. But sklearn handles that by itself. So no need to do it for now.

In [40]:
from sklearn import svm

In [43]:
svm_classifier = svm.SVC(kernel='linear', C=1.0)
svm_classifier.fit(X_train, y_train)  ## do training 
svm_classifier.score(X_train, y_train)

1.0

- So we got 100% accuracy on training data.

In [45]:
svm_classifier.coef_.shape , svm_classifier.intercept_.shape

((3, 3072), (3,))

- As we have 3 unique classes. So we got $^3C_2=3$ sets of weights and bias. Here we got 3 theta which are trained on 3classifiers for 3 pairs: to classify=> (0&1), (0&2), (1&2)
- And all these 3 thetas have 3072 weights and one bias term

## Testing:
- To get testing data, we need to again use Path.glob(<regex>) method to itereate over all images and image_ids are given in testing data. We can extract image_ids also from image name. But order of images should match with order of ids given in test.csv in submission.
- So we will get ids from Path and then make them in order given in test.csv

In [49]:
p = Path("./Test/Images")
files = p.glob("*.jpg")
test_images = []
test_ids = []  # to store ids of corresponding images
for file in files:
    test_img = image.load_img(file, target_size=(32,32)) ## to load ith image
    img_array = image.img_to_array(test_img)  ## convert image to array
    test_images.append(img_array)
    
    img_id = str(file).split("/")[-1]
    test_ids.append(img_id)

- Now convert intensity range from 0-255 to 0.0-1.0 and also convert these lists to numpy arrays

In [57]:
test_data = np.array(test_images, dtype="float32")/255.0
test_ids = np.array(test_ids)
test_data.shape, test_ids.shape

((123, 32, 32, 3), (123,))

In [65]:
## flatten images from 3d-->1d
new_test_data = test_data.reshape((test_data.shape[0],-1))
new_test_data.shape, test_ids.shape

((123, 3072), (123,))

- Add ids as 1st feature to make a total of 3073 features

In [66]:
new_test_data = np.hstack((test_ids.reshape((-1,1)),new_test_data))
new_test_data.shape

(123, 3073)

In [67]:
## convert above to dataframe
new_test_data = pd.DataFrame(new_test_data)
new_test_data.head()

,0,1,2,3,4,5,6,7,8,9,...,3063,3064,3065,3066,3067,3068,3069,3070,3071,3072
0,2023.jpg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2211.jpg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2223.jpg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1138.jpg,0.42745098,0.5411765,0.6509804,0.42745098,0.5411765,0.6509804,0.42745098,0.5411765,0.6509804,...,0.11372549,0.050980393,0.15294118,0.11372549,0.050980393,0.15294118,0.11372549,0.050980393,0.15294118,0.11372549
4,2228.jpg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


- Now we need to get image_ids from testing data and make the order same as given in that.

In [68]:
df = pd.read_csv("./Test/test.csv")
df.head()

,ImageId
0,1106.jpg
1,2236.jpg
2,1131.jpg
3,2221.jpg
4,2011.jpg


- Now we can merge with left_on=ImageId to make order same as given in test.csv 

In [69]:
df2 = pd.merge(df, new_test_data, left_on="ImageId", right_on=0)
df2.head()

,ImageId,0,1,2,3,4,5,6,7,8,...,3063,3064,3065,3066,3067,3068,3069,3070,3071,3072
0,1106.jpg,1106.jpg,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,2236.jpg,2236.jpg,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1131.jpg,1131.jpg,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,2221.jpg,2221.jpg,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.99607843,0.8117647,0.09019608,0.0,0.5137255,0.78039217,0.8627451,0.10980392,0.13725491
4,2011.jpg,2011.jpg,0.5137255,0.5058824,0.5176471,0.5529412,0.54509807,0.54901963,0.5294118,0.52156866,...,0.5803922,0.44313726,0.42352942,0.4,0.6,0.5921569,0.59607846,0.3529412,0.31764707,0.32941177


- Now divide this is our X_test from 3rd column onwards. Do training on this and then get results.

In [70]:
X_test = df2.values[:,2:]
X_test.shape

(123, 3072)

In [71]:
y_pred = svm_classifier.predict(X_test)
y_pred.shape

(123,)

- Store output in output file in format given in test/submission.csv file. 

In [79]:
output = pd.DataFrame(df2.values[:,0], columns=["ImageId"])  ## convert ImageIds to dataframe
output["NameOfPokemon"] = y_pred  ## add another column which has outputs
output.head()

,ImageId,NameOfPokemon
0,1106.jpg,Bulbasaur
1,2236.jpg,Charmander
2,1131.jpg,Bulbasaur
3,2221.jpg,Charmander
4,2011.jpg,Pikachu


In [81]:
output.to_csv("Output.csv", index=False)

#### Accuracy: 94%
- I tried to improve accuracy. I did gridSearch which gave me kernel='rbf' and c=5.0 as best hyperparameter, using which my accuracy got 95%
- Then I tried to change img size from 32,32 to 64,64 but then my accuracy decreases back to 94%. Using 50x50 image also got 94% accuracy: This is bcoz for large size of images, although we have more information, but size of theta will be very large: means there will be a large number of features for seperating hyperplane which lowers the accuracy(which we studied in PCA and feature engineering.)